# SPLADE for Portuguese

Inspired by https://github.com/naver/splade

## Libraries installation

In [1]:
!pip install git+https://github.com/naver/splade.git -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 51.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 59.8 MB/s eta 0:00:00


In [2]:
!pip install hydra-core --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=2da34cc91c2c5b07858b8a1f59a028c8b81a1c30ecca4c6c6626e3bed5338d0b
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.8
    Uninstalling antlr4-python3-runtime-4.8:
      Successfully uninstalled antlr4-python3-runtime-4.8
  Attempting uninstall: omegaconf
    Found existing installation: omegaconf

In [3]:
!pip install pytrec_eval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=293476 sha256=36c35403a8c499b4a1cbbbff2dd33896352736ea19bb2e02f2d46064c22dcc66
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval


## File system preparation

In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [89]:
main_dir = "/content/gdrive/MyDrive/Unicamp-projeto-final"

In [19]:
!git clone https://github.com/naver/splade.git {main_dir}/splade

Cloning into '/content/gdrive/MyDrive/Unicamp-projeto-final/splade'...
remote: Enumerating objects: 589, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 589 (delta 57), reused 80 (delta 45), pack-reused 467
Receiving objects: 100% (589/589), 3.12 MiB | 12.22 MiB/s, done.
Resolving deltas: 100% (299/299), done.


 ## Configuration setup

In [141]:
import os

CONFIG_NAME = None
CONFIG_PATH = "gdrive/MyDrive/Unicamp-projeto-final/splade/conf"

##############################################################
# Provide (as env var), either:
# * 'SPLADE_CONFIG_NAME', this config in splade/conf will be used
# * or 'SPLADE_CONFIG_FULLPATH' (full path, from an exp, such as '/my/path/to/exp/config.yaml'

# if nothing is provided, 'config_default' is used
##############################################################

assert sum([v in os.environ.keys() for v in ["SPLADE_CONFIG_NAME", "SPLADE_CONFIG_FULLPATH"]]) <= 1

if "SPLADE_CONFIG_NAME" in os.environ.keys():
    CONFIG_NAME = os.environ["SPLADE_CONFIG_NAME"]
elif "SPLADE_CONFIG_FULLPATH" in os.environ.keys():
    CONFIG_FULLPATH = os.environ["SPLADE_CONFIG_FULLPATH"]
    CONFIG_PATH, CONFIG_NAME = os.path.split(CONFIG_FULLPATH)
else:
    #CONFIG_NAME = "config_default"
    CONFIG_NAME = "config_default_pt"

if ".yaml" in CONFIG_NAME:
    CONFIG_NAME = CONFIG_NAME.split(".yaml")[0]


In [142]:
CONFIG_NAME

'config_default_pt'

In [143]:
CONFIG_PATH

'gdrive/MyDrive/Unicamp-projeto-final/splade/conf'

In [144]:
import os

import hydra
import torch
from omegaconf import DictConfig, open_dict
from torch.utils import data

from splade.datasets.dataloaders import CollectionDataLoader, SiamesePairsDataLoader, DistilSiamesePairsDataLoader
from splade.datasets.datasets import PairsDatasetPreLoad, DistilPairsDatasetPreLoad, MsMarcoHardNegatives, \
    CollectionDatasetPreLoad
from splade.losses.regularization import init_regularizer, RegWeightScheduler
from splade.models.models_utils import get_model
from splade.optim.bert_optim import init_simple_bert_optim
from splade.tasks.transformer_evaluator import SparseApproxEvalWrapper
from splade.tasks.transformer_trainer import SiameseTransformerTrainer
from splade.utils.utils import set_seed, restore_model, get_initialize_config, get_loss, set_seed_from_config

In [145]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
import hydra

In [146]:
# Initialize Hydra
hydra.core.global_hydra.GlobalHydra.instance().clear()

initialize(config_path=CONFIG_PATH)

# Load the configuration
config = compose(config_name=CONFIG_NAME)

# Convert the configuration to a dictionary
exp_dict = OmegaConf.to_container(config, resolve=True)

<ipython-input-146-b8d155ab2a17>:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path=CONFIG_PATH)
/usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config_default_pt': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [147]:
type(exp_dict)

dict

In [148]:
exp_dict = DictConfig(exp_dict)

In [149]:
type(exp_dict)

omegaconf.dictconfig.DictConfig

## Training loop

In [150]:
def train(exp_dict: DictConfig):
    exp_dict, config, init_dict, _ = get_initialize_config(exp_dict, train=True)
    model = get_model(config, init_dict)
    random_seed = set_seed_from_config(config)

    optimizer, scheduler = init_simple_bert_optim(model, lr=config["lr"], warmup_steps=config["warmup_steps"],
                                                  weight_decay=config["weight_decay"],
                                                  num_training_steps=config["nb_iterations"])

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    ################################################################
    # CHECK IF RESUME TRAINING
    ################################################################
    iterations = (1, config["nb_iterations"] + 1)  # tuple with START and END
    regularizer = None
    if os.path.exists(os.path.join(config["checkpoint_dir"], "model_ckpt/model_last.tar")):
        print("@@@@ RESUMING TRAINING @@@")
        print("WARNING: change seed to change data order when restoring !")
        set_seed(random_seed + 666)
        if device == torch.device("cuda"):
            ckpt = torch.load(os.path.join(config["checkpoint_dir"], "model_ckpt/model_last.tar"))
        else:
            ckpt = torch.load(os.path.join(config["checkpoint_dir"], "model_ckpt/model_last.tar"), map_location=device)
        print("starting from step", ckpt["step"])
        print("{} remaining iterations".format(iterations[1] - ckpt["step"]))
        iterations = (ckpt["step"] + 1, config["nb_iterations"])
        restore_model(model, ckpt["model_state_dict"])
        optimizer.load_state_dict(ckpt["optimizer_state_dict"])
        if device == torch.device("cuda"):
            for state in optimizer.state.values():
                for k, v in state.items():
                    if torch.is_tensor(v):
                        state[k] = v.cuda()
        scheduler.load_state_dict(ckpt["scheduler_state_dict"])
        if "regularizer" in ckpt:
            print("loading regularizer")
            regularizer = ckpt.get("regularizer", None)

    if torch.cuda.device_count() > 1:
        print(" --- use {} GPUs --- ".format(torch.cuda.device_count()))
        model = torch.nn.DataParallel(model)
    model.to(device)

    loss = get_loss(config)

    # initialize regularizer dict
    if "regularizer" in config and regularizer is None:  # else regularizer is loaded
        output_dim = model.module.output_dim if hasattr(model, "module") else model.output_dim
        regularizer = {"eval": {"L0": {"loss": init_regularizer("L0")},
                                "sparsity_ratio": {"loss": init_regularizer("sparsity_ratio",
                                                                            output_dim=output_dim)}},
                       "train": {}}
        if config["regularizer"] == "eval_only":
            # just in the case we train a model without reg but still want the eval metrics like L0
            pass
        else:
            for reg in config["regularizer"]:
                temp = {"loss": init_regularizer(config["regularizer"][reg]["reg"]),
                        "targeted_rep": config["regularizer"][reg]["targeted_rep"]}
                d_ = {}
                if "lambda_q" in config["regularizer"][reg]:
                    d_["lambda_q"] = RegWeightScheduler(config["regularizer"][reg]["lambda_q"],
                                                        config["regularizer"][reg]["T"])
                if "lambda_d" in config["regularizer"][reg]:
                    d_["lambda_d"] = RegWeightScheduler(config["regularizer"][reg]["lambda_d"],
                                                        config["regularizer"][reg]["T"])
                temp["lambdas"] = d_  # it is possible to have reg only on q or d if e.g. you only specify lambda_q
                # in the reg config
                # targeted_rep is just used to indicate which rep to constrain (if e.g. the model outputs several
                # representations)
                # the common case: model outputs "rep" (in forward) and this should be the value for this targeted_rep
                regularizer["train"][reg] = temp

    # fix for current in batch neg losses that break on last batch
    if config["loss"] in ("InBatchNegHingeLoss", "InBatchPairwiseNLL"):
        drop_last = True
    else:
        drop_last = False

    if exp_dict["data"].get("type", "") == "triplets":
        data_train = PairsDatasetPreLoad(data_dir=exp_dict["data"]["TRAIN_DATA_DIR"])
        train_mode = "triplets"
    elif exp_dict["data"].get("type", "") == "triplets_with_distil":
        data_train = DistilPairsDatasetPreLoad(data_dir=exp_dict["data"]["TRAIN_DATA_DIR"])
        train_mode = "triplets_with_distil"
    elif exp_dict["data"].get("type", "") == "hard_negatives":
        data_train = MsMarcoHardNegatives(
            dataset_path=exp_dict["data"]["TRAIN"]["DATASET_PATH"],
            document_dir=exp_dict["data"]["TRAIN"]["D_COLLECTION_PATH"],
            query_dir=exp_dict["data"]["TRAIN"]["Q_COLLECTION_PATH"],
            qrels_path=exp_dict["data"]["TRAIN"]["QREL_PATH"])
        train_mode = "triplets_with_distil"
    else:
        raise ValueError("provide valid data type for training")

    val_loss_loader = None  # default
    if "VALIDATION_SIZE_FOR_LOSS" in exp_dict["data"]:
        print("initialize loader for validation loss")
        print("split train, originally {} pairs".format(len(data_train)))
        data_train, data_val = torch.utils.data.random_split(data_train, lengths=[
            len(data_train) - exp_dict["data"]["VALIDATION_SIZE_FOR_LOSS"],
            exp_dict["data"]["VALIDATION_SIZE_FOR_LOSS"]])
        print("train: {} pairs ~~ val: {} pairs".format(len(data_train), len(data_val)))
        if train_mode == "triplets":
            val_loss_loader = SiamesePairsDataLoader(dataset=data_val, batch_size=config["eval_batch_size"],
                                                     shuffle=False,
                                                     num_workers=4,
                                                     tokenizer_type=config["tokenizer_type"],
                                                     max_length=config["max_length"], drop_last=drop_last)
        elif train_mode == "triplets_with_distil":
            val_loss_loader = DistilSiamesePairsDataLoader(dataset=data_val, batch_size=config["eval_batch_size"],
                                                           shuffle=False,
                                                           num_workers=4,
                                                           tokenizer_type=config["tokenizer_type"],
                                                           max_length=config["max_length"], drop_last=drop_last)
        else:
            raise NotImplementedError

    if train_mode == "triplets":
        train_loader = SiamesePairsDataLoader(dataset=data_train, batch_size=config["train_batch_size"], shuffle=True,
                                              num_workers=4,
                                              tokenizer_type=config["tokenizer_type"],
                                              max_length=config["max_length"], drop_last=drop_last)
    elif train_mode == "triplets_with_distil":
        train_loader = DistilSiamesePairsDataLoader(dataset=data_train, batch_size=config["train_batch_size"],
                                                    shuffle=True,
                                                    num_workers=4,
                                                    tokenizer_type=config["tokenizer_type"],
                                                    max_length=config["max_length"], drop_last=drop_last)
    else:
        raise NotImplementedError

    val_evaluator = None
    if "VALIDATION_FULL_RANKING" in exp_dict["data"]:
        with open_dict(config):
            config["val_full_rank_qrel_path"] = exp_dict["data"]["VALIDATION_FULL_RANKING"]["QREL_PATH"]
        full_ranking_d_collection = CollectionDatasetPreLoad(
            data_dir=exp_dict["data"]["VALIDATION_FULL_RANKING"]["D_COLLECTION_PATH"], id_style="row_id")
        full_ranking_d_loader = CollectionDataLoader(dataset=full_ranking_d_collection,
                                                     tokenizer_type=config["tokenizer_type"],
                                                     max_length=config["max_length"],
                                                     batch_size=config["eval_batch_size"],
                                                     shuffle=False, num_workers=4)
        full_ranking_q_collection = CollectionDatasetPreLoad(
            data_dir=exp_dict["data"]["VALIDATION_FULL_RANKING"]["Q_COLLECTION_PATH"], id_style="row_id")
        full_ranking_q_loader = CollectionDataLoader(dataset=full_ranking_q_collection,
                                                     tokenizer_type=config["tokenizer_type"],
                                                     max_length=config["max_length"], batch_size=1,
                                                     # TODO fix: bs currently set to 1
                                                     shuffle=False, num_workers=4)
        val_evaluator = SparseApproxEvalWrapper(model,
                                                config={"top_k": exp_dict["data"]["VALIDATION_FULL_RANKING"]["TOP_K"],
                                                        "out_dir": os.path.join(config["checkpoint_dir"],
                                                                                "val_full_ranking")
                                                        },
                                                collection_loader=full_ranking_d_loader,
                                                q_loader=full_ranking_q_loader,
                                                restore=False)

    # #################################################################
    # # TRAIN
    # #################################################################
    print("+++++ BEGIN TRAINING +++++")
    trainer = SiameseTransformerTrainer(model=model, iterations=iterations, loss=loss, optimizer=optimizer,
                                        config=config, scheduler=scheduler,
                                        train_loader=train_loader, validation_loss_loader=val_loss_loader,
                                        validation_evaluator=val_evaluator,
                                        regularizer=regularizer)
    trainer.train()


In [151]:
exp_dict

{'config': {'lr': 2e-05, 'seed': 123, 'gradient_accumulation_steps': 1, 'weight_decay': 0.01, 'validation_metrics': ['MRR@10', 'recall@100', 'recall@200', 'recall@500'], 'pretrained_no_yamlconfig': False, 'nb_iterations': 10, 'train_batch_size': 6, 'eval_batch_size': 8, 'index_retrieve_batch_size': 6, 'record_frequency': 3, 'train_monitoring_freq': 2, 'warmup_steps': 5, 'max_length': 10, 'fp16': False, 'augment_pairs': 'in_batch_negatives', 'matching_type': 'splade', 'monitoring_ckpt': 'loss', 'loss': 'InBatchPairwiseNLL', 'regularizer': {'FLOPS': {'lambda_q': 0.0005, 'lambda_d': 0.0003, 'T': 3, 'targeted_rep': 'rep', 'reg': 'FLOPS'}}, 'tokenizer_type': 'distilbert-base-uncased', 'top_k': 5, 'threshold': 0.4, 'eval_metric': [['mrr_10', 'recall']], 'checkpoint_dir': 'splade_pt_checkpoint', 'index_dir': 'splade_pt_index', 'out_dir': 'splade_pt_out'}, 'data': {'type': 'triplets', 'TRAIN_DATA_DIR': '/content/gdrive/MyDrive/Unicamp-projeto-final/splade/data/toy_data/triplets', 'VALIDATION_S

In [152]:
train(exp_dict)

config:
  lr: 2.0e-05
  seed: 123
  gradient_accumulation_steps: 1
  weight_decay: 0.01
  validation_metrics:
  - MRR@10
  - recall@100
  - recall@200
  - recall@500
  pretrained_no_yamlconfig: false
  nb_iterations: 10
  train_batch_size: 6
  eval_batch_size: 8
  index_retrieve_batch_size: 6
  record_frequency: 3
  train_monitoring_freq: 2
  warmup_steps: 5
  max_length: 10
  fp16: false
  augment_pairs: in_batch_negatives
  matching_type: splade
  monitoring_ckpt: loss
  loss: InBatchPairwiseNLL
  regularizer:
    FLOPS:
      lambda_q: 0.0005
      lambda_d: 0.0003
      T: 3
      targeted_rep: rep
      reg: FLOPS
  tokenizer_type: distilbert-base-uncased
  top_k: 5
  threshold: 0.4
  eval_metric:
  - - mrr_10
    - recall
  checkpoint_dir: splade_pt_checkpoint
  index_dir: splade_pt_index
  out_dir: splade_pt_out
data:
  type: triplets
  TRAIN_DATA_DIR: /content/gdrive/MyDrive/Unicamp-projeto-final/splade/data/toy_data/triplets
  VALIDATION_SIZE_FOR_LOSS: 20
  VALIDATION_FULL_RAN

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0it [00:00, ?it/s]

initialize loader for validation loss
split train, originally 100 pairs
train: 80 pairs ~~ val: 20 pairs


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/3 [00:01<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


+batch_loss_iter10: 98.836
======= TRAINING DONE =======
took about 0.09917896290620168 hours
